In [94]:
import csv
import pandas_datareader as web
import yfinance as yf
import sys 
Symboles=[]
"""mettre en liste les symboles du CAC40??"""
with open('symboles.csv') as file:
    reader = csv.reader(file, delimiter=',')
    line = 0
    for i in reader:
        if line ==0:
            
            #print(f'Nom des colonnes : {", ".join(i)}')
            line+=1
        else:
           # print(i[0]," :: ",i[1])
            Symboles.append([i[0],i[1]])
            line+=1
    #print(f"Il y a en tout : {line} lignes")
#help(list.remove)
print( "Nombre de symboles:", len(Symboles))
Symboles.remove(['club','CLUB'])
Symboles.remove(['valx','VALX'])
Symboles.remove(['plpm', 'PLPM'])
Symboles.remove(['scac', 'SCAC'])
#Symboles.remove
#print(Symboles)

Nombre de symboles: 7195


In [118]:
with open('dataset.csv','w', newline='') as wfile:
    writer = csv.writer(wfile)
    
    Data={}
    #help(web.DataReader)

    for s in Symboles:
    
        """values = web.DataReader(s[1], 'yahoo','1/1/2021')

        sf = yf.Ticker(s[1])
        Data[s[1]]=[values,sf.info]"""
        try:
            values = web.DataReader(s[1], 'yahoo','1/1/2021')
            sf = yf.Ticker(s[1])
            writer.writerow(sf.info)
            Data[s[1]]=[values,sf.info]
            csv
        except web._utils.RemoteDataError as e:
    #e.errno
    #except RemoteDataError as exp :
            print('Unable to read data: {0}'.format(s[1]))
    #continue
    #else: continue
    #RemoteDataError 
    #Data[s[1]]
    #Data.append([values])
        print(s[1])


KELYB
PLYM
ALKS
MEDP
TMO
ACIU
CEY
Unable to read data: LNC-WS
LNC-WS
LINC
GLO
SPOK
Unable to read data: NRE
NRE
KIO
NXPI
GIL


KeyboardInterrupt: 

In [53]:
#print(Data)

#web.DataReader('SCAC', 'yahoo','2/2/2021')


In [15]:
import yfinance as yf
sf = yf.Ticker('WBT')
T = sf.info
tn =''
for name , t in T.items():
    tn += str(t)+'; '
    #print(f"{name} New Line: {tn} type: {type((tn))}")
print(tn)

34655; Industrials; 4400; Welbilt, Inc., designs, manufactures, and supplies foodservice equipment for commercial foodservice market worldwide. The company offers commercial upright and undercounter refrigerators and freezers, blast freezers and chillers, and cook-chill systems under the Delfield brand; and walk-in refrigerators, coolers and freezers, and prefabricated cooler and freezer panels under the Kolpak brand. It also provides traditional, combination, convection, conveyor, and rapid-cooking ovens, and range and grill products under the Convotherm, Garland, Lincoln, and Merrychef brands; fryers and frying systems under the Frymaster brand; and steam equipment under the Cleveland brand. In addition, the company offers cafeteria and buffet equipment stations, bins, boxes, warming cabinets, warmers, display and deli cases, and insulated and refrigerated salad and food bars under the Delfield, Merco, and other brand names; beverage dispensers, blended ice machines, ice/beverage dis

In [67]:
"""import yfinance as yf
import csv
#
s="\"[\'"
sf = yf.Ticker('alks')
for name, _ in sf.info.items():
    #name, _ = 
    """if name == 'logo_url':
        s+= name"""
    #else:
    s+= name+'; '
s+= "\'\']\""
    #print(name,'\n\n')
print( s)"""

"['zip; sector; fullTimeEmployees; longBusinessSummary; city; phone; country; companyOfficers; website; maxAge; address1; industry; address2; previousClose; regularMarketOpen; twoHundredDayAverage; trailingAnnualDividendYield; payoutRatio; volume24Hr; regularMarketDayHigh; navPrice; averageDailyVolume10Day; totalAssets; regularMarketPreviousClose; fiftyDayAverage; trailingAnnualDividendRate; open; toCurrency; averageVolume10days; expireDate; yield; algorithm; dividendRate; exDividendDate; beta; circulatingSupply; startDate; regularMarketDayLow; priceHint; currency; regularMarketVolume; lastMarket; maxSupply; openInterest; marketCap; volumeAllCurrencies; strikePrice; averageVolume; priceToSalesTrailing12Months; dayLow; ask; ytdReturn; askSize; volume; fiftyTwoWeekHigh; forwardPE; fromCurrency; fiveYearAvgDividendYield; fiftyTwoWeekLow; bid; tradeable; dividendYield; bidSize; dayHigh; exchange; shortName; longName; exchangeTimezoneName; exchangeTimezoneShortName; isEsgPopulated; gmtOffSe

In [68]:
def get_data(table, nbT):
    nTable = table
    namefile= 'dataset.csv'
    i = 0
    with open (namefile,'w', newline = '') as nfile:
        writer = csv.writer(nfile)
        for s in table:
            try:
                sf= yf.Ticker(s[1])
                print(sf.info.items(), s)
                donnees=''
                for name , d in sf.info.items():
                    #if name == 'logo_url' : donnees+=str(d)
                    donnees+= str(d)+'; '

                if donnees!='':
                    donnees = donnees.replace('\n',' ')
                    writer.writerow([donnees])
                else:
                    nTable.remove(s)
                i+=1
                if (i%100==0):
                    print(f'More: {i}')

            except web._utils.RemoteDataError :
                #print('Unable to read data from symbol {0}'.format(s[1]))
                nTable.remove(s)
            except Exception as e:
                #print('Unable to read data from date {0}'.format('1/1?2021'))
                nTable.remove(s)
    return (nTable, nbT )

In [69]:
x , n = get_data(Table,0)


dict_items([('zip', '02110'), ('sector', 'Real Estate'), ('fullTimeEmployees', 34), ('longBusinessSummary', 'Plymouth Industrial REIT, Inc. is a vertically integrated and self-managed real estate investment trust focused on the acquisition and operation of single and multi-tenant industrial properties located in secondary and select primary markets across the United States. The Company seeks to acquire properties that provide income and growth that enable the Company to leverage its real estate operating expertise to enhance shareholder value through active asset management, prudent property re-positioning and disciplined capital deployment.'), ('city', 'Boston'), ('phone', '617-340-3814'), ('state', 'MA'), ('country', 'United States'), ('companyOfficers', []), ('website', 'http://www.plymouthreit.com'), ('maxAge', 1), ('address1', '260 Franklin Street'), ('industry', 'REIT—Industrial'), ('address2', 'Suite 700'), ('previousClose', 16.85), ('regularMarketOpen', 16.87), ('twoHundredDayA

dict_items([('previousClose', 18.345), ('regularMarketOpen', 18.32), ('twoHundredDayAverage', 18.345), ('trailingAnnualDividendYield', None), ('payoutRatio', None), ('volume24Hr', None), ('regularMarketDayHigh', 18.52), ('navPrice', 18.35), ('averageDailyVolume10Day', 0), ('totalAssets', 14845205), ('regularMarketPreviousClose', 18.345), ('fiftyDayAverage', 18.345), ('trailingAnnualDividendRate', None), ('open', 18.32), ('toCurrency', None), ('averageVolume10days', 0), ('expireDate', None), ('yield', 0.027), ('algorithm', None), ('dividendRate', None), ('exDividendDate', None), ('beta', None), ('circulatingSupply', None), ('startDate', None), ('regularMarketDayLow', 18.52), ('priceHint', 2), ('currency', 'USD'), ('regularMarketVolume', 27), ('lastMarket', None), ('maxSupply', None), ('openInterest', None), ('marketCap', None), ('volumeAllCurrencies', None), ('strikePrice', None), ('averageVolume', 0), ('priceToSalesTrailing12Months', None), ('dayLow', 18.52), ('ask', 0), ('ytdReturn', 

In [52]:
print(x)

[['plym', 'PLYM'], ['alks', 'ALKS'], ['valx', 'VALX'], ['medp', 'MEDP']]


In [45]:
import quandl
import numpy as np
from sklearn .linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
import datetime
import pandas_datareader as web
import pandas 

In [30]:
start = datetime.datetime(2021,1,1)
end = datetime.datetime(2021,3,30)
CAC40= [EU:AC, EU:AI, EU:ALU, EU:ALO, EU:MT, EU:CS, EU:BNP, EU:EN, EU:CAP, EU:CA, EU:ACA, EU:BN, EU:DEXB, EU:EAD, EU:EDF, EU:EI, EU:FTE, EU:GSZ, EU:OR, EU:LG, EU:LR, EU:MMB, EU:MC, EU:ML, EU:RI, EU:UG, EU:PP, EU:RNO, EU:SGO, EU:SAN, EU:SU, EU:GLE, EU:STM, EU:TEC, EU:FP, EU:UL, EU:VK, EU:VIE, EU:DG, EU:VIV]

In [112]:
df = web.DataReader('AMZN','yahoo',start, end)
print(type(df))
df = pandas.DataFrame(df)
print(type(df))
#df.tail()
forecast_out = 30 #nb jours
df=df[['Adj Close']]

df['Prediction']= df.shift(-forecast_out)
df

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


,Adj Close,Prediction
Date,,
2021-01-04,3186.629883,3308.639893
2021-01-05,3218.510010,3328.229980
2021-01-06,3138.379883,3249.899902
2021-01-07,3162.159912,3180.739990
2021-01-08,3182.699951,3194.500000
2021-01-11,3114.209961,3159.530029
2021-01-12,3120.830078,3057.159912
2021-01-13,3165.889893,3092.929932
2021-01-14,3127.469971,3146.139893


In [113]:
"""forecast_out = 30 #nb jours
df['Prediction']= df[['Adj. Close']].shift(-forecast_out)
print(df.tail())"""

"forecast_out = 30 #nb jours\ndf['Prediction']= df[['Adj. Close']].shift(-forecast_out)\nprint(df.tail())"

In [114]:
X = np.array(df.drop(['Prediction'],1))
X = X[:-forecast_out]
y = np.array(df['Prediction'])
y = y[:-forecast_out]
print(X,'\n',y)

[[3186.62988281]
 [3218.51000977]
 [3138.37988281]
 [3162.15991211]
 [3182.69995117]
 [3114.20996094]
 [3120.83007812]
 [3165.88989258]
 [3127.4699707 ]
 [3104.25      ]
 [3120.76000977]
 [3263.37988281]
 [3306.98999023]
 [3292.22998047]
 [3294.        ]
 [3326.12988281]
 [3232.58007812]
 [3237.62011719]
 [3206.19995117]
 [3342.87988281]
 [3380.        ]
 [3312.5300293 ]
 [3331.        ]
 [3352.14990234]
 [3322.93994141]
 [3305.        ]
 [3286.58007812]
 [3262.12988281]
 [3277.70996094]
 [3268.94995117]] 
 [3308.63989258 3328.22998047 3249.89990234 3180.73999023 3194.5
 3159.5300293  3057.15991211 3092.92993164 3146.13989258 3094.5300293
 3005.         2977.57006836 3000.45996094 2951.94995117 3062.85009766
 3057.63989258 3113.59008789 3089.48999023 3081.67993164 3091.86010742
 3135.72998047 3027.98999023 3074.95996094 3110.87011719 3137.5
 3087.07006836 3046.26000977 3052.0300293  3075.72998047 3055.29003906]


In [115]:
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size = .2)

In [138]:
svr_rbf = SVR(kernel ='rbf', C =1e3, gamma = .1)
C = np.exp(-1*np.arange(0,15))
for c in C:
    svr_rbf = SVR(kernel ='rbf', C =c, gamma = 20)
    svr_rbf.fit(x_train, y_train)
    svm_confidence = svr_rbf.score(x_test, y_test)
    print(f"score pour c = {c} svm: {svm_confidence}")
sve_rbf = SVR(kernel ='rbf', C =.00091, gamma = .1)

score pour c = 1.0 svm: -0.018358654371158023
score pour c = 0.36787944117144233 svm: -0.018358654371158023
score pour c = 0.1353352832366127 svm: -0.018358654371158023
score pour c = 0.049787068367863944 svm: -0.018358654371158023
score pour c = 0.01831563888873418 svm: -0.018358654371158023
score pour c = 0.006737946999085467 svm: -0.018358654371158023
score pour c = 0.0024787521766663585 svm: -0.018358654371158023
score pour c = 0.0009118819655545162 svm: -0.018358654371158023
score pour c = 0.00033546262790251185 svm: -0.018358654371158023
score pour c = 0.00012340980408667956 svm: -0.018358654371158023
score pour c = 4.5399929762484854e-05 svm: -0.018358654371158023
score pour c = 1.670170079024566e-05 svm: -0.018358654371158023
score pour c = 6.14421235332821e-06 svm: -0.018358654371158023
score pour c = 2.2603294069810542e-06 svm: -0.018358654371158023
score pour c = 8.315287191035679e-07 svm: -0.018358654371158023


In [131]:
svm_confidence = svr_rbf.score(x_test, y_test)
print("score svm: ", svm_confidence)

score svm:  -0.018358647810280848
